In [1]:
import os

In [2]:
# TF_ODA2_PATH = r'..\..\..\1_playdataInstalls\TF_oda2\models\research'

BASE_PATH = r'..\..\..\1.Data\object_detection_workspace\workspace' #  작업시 생기는 파일들을 저장할 root디렉토리.
SCRIPT_PATH = r'..\..\..\1.Data\object_detection_workspace\scripts' # utility python script들이 저장된 디렉토리.
TF_OD_API_PATH = r'..\..\..\1_playdataInstalls\TF_oda2\models' # Tensorflow object detection api 설치 경로.

IMAGE_PATH = os.path.join(BASE_PATH, 'images') # image data들, annotation 파일이 저장된 디렉토리.

LABEL_MAP_PATH = os.path.join(BASE_PATH, 'labelmap') # Label map파일이 저장된 디렉토리.
LABEL_MAP_FILE_PATH = os.path.join(LABEL_MAP_PATH, 'label_map.pbtxt') # Label_map파일 경로

TF_RECORD_PATH = os.path.join(BASE_PATH, 'tfrecord') # TFRecord파일들을 저장할 경로.

MODEL_PATH = os.path.join(BASE_PATH, 'model') # pretrained 모델 fine tuning한 모델, weight(ckpt), pipeline.config를 저장할 경로.
CHECK_POINT_PATH = os.path.join(MODEL_PATH, 'checkpoint') # 학습도중에 중간 중간 저장되는 weight
EXPORT_MODEL_PATH = os.path.join(MODEL_PATH, 'export_model') # fine tuning한 최종 모델을 저장할 경로
PIPELINE_CONFIG_PATH = os.path.join(MODEL_PATH, 'pipeline.config') # pipeline.config(설정파일)의 경로.

PRE_TRAINED_MODEL_PATH = os.path.join(BASE_PATH, 'pre_trained_model') # 전이학습 시킬 model을 저장할 경로.

In [6]:
PRE_TRAINED_MODEL_PATH

'..\\..\\..\\1.Data\\object_detection_workspace\\workspace\\pre_trained_model'

In [4]:
os.environ['PYTHONPATH']

'.;C:\\Users\\mein0\\1_playdataInstalls\\TF_oda2\\models;C:\\Users\\mein0\\1_playdataInstalls\\TF_oda2\\models\\research;'

In [36]:
pip uninstall opencv-python

^C
Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [4]:
# pipeline.config에 맞춰서 추출한 모델을 바탕으로 모델을 생성

# pipeline.config를 조회
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)
# print(config)
#config 정보를 넣어서 모델생성
detection_model = model_builder.build(model_config=config['model'], is_training=False)

# 모델에 학습시킨 checkpoint(weight)를 주입
# checkpoint 조회
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECK_POINT_PATH,'ckpt-21')).expect_partial()

In [5]:
# detection을 실행하는 함수

# 순전파 처리 함수에 @tf.function decorator를 선언하면 실행 속도가 빨라진다.
@tf.function
def detect_func(image):
    """
    매개변수로 object detection을 수행할 대상 image(Tensor)를 받아서 detection처리.
    1. preprocessing(전처리): resize, normalization 작업
    2. detection(inference-추론)
    3. detection결과를 postprocessing: Non Maximum Suppression
    4. postprocessing한 결과를 반환.
    """
    # 1. preprocessing
    image, shapes = detection_model.preprocess(image)
    # 2. 추론
    predict_dic = detection_model.predict(image,shapes)
    # 3. post processing
    result = detection_model.postprocess(predict_dic, shapes)
    # 4. 반환
    return result

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_FILE_PATH)
print(type(category_index))
category_index

<class 'dict'>


{1: {'id': 1, 'name': 'one'},
 2: {'id': 2, 'name': 'two'},
 3: {'id': 3, 'name': 'three'},
 4: {'id': 4, 'name': 'four'},
 5: {'id': 5, 'name': 'five'}}

In [10]:
# 웹캠으로 부터 이미지를 받아서 추론한 결과를 화면에 보여주기.
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print('webcam failed')
# print(cap.isOpened())
# 웹캠의 width/height 조회
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

True


In [11]:
while True:
    ret, frame = cap.read() # 한 frame 읽기.
    if not ret:
        print("이미지를 읽지 못함")
        break
        
    frame = cv2.flip(frame, 1) # 좌우 반전.
    
    # BGR => RGB (모델이 학습할때 RGB 모드로 학습했기 때문에 같은형식으로 변환)
    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # 0번축을 늘린후, Tensor로 변환
    input_tensor = tf.convert_to_tensor(image_np[np.newaxis, ...], dtype=tf.float32)
    
    # 추론
    post_detection = detect_func(input_tensor) # 전처리->추론->후처리
    
    
    num_detections = int(post_detection.pop('num_detections'))
    # 추론한 결과들을 num_detections 개수(detection한 물체의 개수)만큼의 값만 남긴다. 결과가 Tensor로 반환되는 것을 ndarray로 반환.
    detections = { key:value[0, :num_detections].numpy() for key, value in post_detection.items()}
    # 새로 구성한 결과 dictionary(detections)에 num_detections 값을 추가
    detections['num_detections'] = num_detections
    # detection_classes는 검출한 box의 class값을 label encoding된 값으로 가진다. float32로 반환되는 것을 int로 변환 처리
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    
    MIN_CONF_THRESH =0.5 # 물체가 있을 confidence score가 0.5이상인 bounding box만 나오도록 하겠다.
    image_np_with_detection = image_np.copy() # detection한 원본 이미지의 카피본을 생성.
    img = viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detection, # 추론한 원본이미지,
        detections['detection_boxes'], # bounding box좌표
        detections['detection_classes'] + 1, # bounding box내의 물체 index(class확률에서 0은 첫번째 label, label map의 id는 1부터 시작하기 때문에 + 1을 해준다.)
        detections['detection_scores'], # bounding box내에 물체가 있을 확률(confidence score)
        category_index,
        use_normalized_coordinates = True, # bounding box의 좌표들이 normalize되었는지 여부
        max_boxes_to_draw=100, # 최대 몇개의 박스를 칠 것인지(default:20)
        min_score_thresh=MIN_CONF_THRESH # confidence score가 얼마 이상인 bounding box만 나오도록 하겠다.
    )
    # 결과 image를 RGB->BGR
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # 화면에 출력
    cv2.imshow('frame', img)
    if cv2.waitKey(1) > 0:
        break

cap.release()
cv2.destroyAllWindows()

## post processing  결과 확인

In [13]:
img = cv2.imread('five_a.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img[np.newaxis, ...]
input_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

In [14]:
post_detection = detect_func(input_tensor)

In [16]:
post_detection.keys()

dict_keys(['detection_boxes', 'detection_scores', 'detection_classes', 'num_detections', 'raw_detection_boxes', 'raw_detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices'])

In [19]:
post_detection['num_detections'].numpy() # tensor => ndarray
# num_detections: 후처리로 최종 결과로 나온 bounding box의 개수. 
#전체 bounding box에서 confidence score순으로 내림 차순한 뒤 NMS를 거쳐서 최종적으로 남은 bounding box의 개수

array([100.], dtype=float32)

In [21]:
post_detection['detection_boxes'].shape
# [1, 100, 4] =>[추론한이미지개수, num_detections, 좌표(x,y,w,h)]

TensorShape([1, 100, 4])

In [22]:
# bounding box(bbox)에 물체가 있을 확률 = confidence score
post_detection['detection_scores'].shape
# [1, 100], [추론한이미지개수, num_detections]

TensorShape([1, 100])

In [24]:
post_detection['detection_scores'][0,:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1.        , 0.03945891, 0.0358245 , 0.02451506, 0.02186941,
       0.01868566, 0.01674229, 0.01609688, 0.01506179, 0.01422502],
      dtype=float32)>

In [26]:
post_detection['detection_boxes'][0,0]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.3586255 , 0.13050056, 0.9508877 , 0.6253593 ], dtype=float32)>

In [36]:

post_detection['detection_scores'][0,0]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [33]:
post_detection['detection_classes'][0,0] +1

<tf.Tensor: shape=(), dtype=float32, numpy=5.0>

In [30]:
# 각 클라스별 확률
post_detection['detection_multiclass_scores'].shape

TensorShape([1, 100, 6])

In [37]:
post_detection['detection_multiclass_scores'][0,0]

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([9.4400661e-05, 8.1133585e-05, 1.5478390e-02, 6.2484560e-03,
       2.4515057e-02, 1.0000000e+00], dtype=float32)>

In [38]:
post_detection['raw_detection_boxes'].shape

TensorShape([1, 12804, 4])